# Prologue
Tensorflow（下稱TF）不易編寫的原因在於：

1. 其編程不像 Python 隨寫隨得 (涉及 computational graph)
3. 官方說明文件和官方教學整合得不夠完善
2. API 不夠方便，例如沒有 Leaky ReLU、沒有 CNN layer 可用。你得自己寫。
2. 高階操作可能被隱藏起來(公開的API列表裡面沒講)，例如
```python  
from tensorflow.python.training import moving_averages
```  
  
Tensorflow 官方提供了兩個版本的教學  
[官網版教學](https://www.tensorflow.org/versions/master/tutorials/index.html)  
這個教學講的是 Tensorflow 的用法。  
此教學分成有很多有趣的實作，  
從最簡單的 MNIST (數字辨識)、CIFAR10 (物件辨識)，乃至word2vec 以及機器翻譯都有。  
有興趣的話也可以去看他的Deep Dream。(上述全部都有提供code)  
其實如果真的學不起來也沒關係。直接修改他的 code 讓你的 task 可以跑就好了。  

[Udacity短課程](https://classroom.udacity.com/courses/ud730/)  
分四個小節講解：基本概念、DNN、CNN、RNN  
本課程的精隨在於他的作業  
作業是修改課程提供的範例code，如果熟 Tensorflow的話，改起來不算太難。  

所有的教學都一定要配~~溫開水~~[API](https://www.tensorflow.org/versions/master/api_docs/python/index.html) 服用

關於 Tensorflow 的安裝，請參考其[Github repository](https://github.com/tensorflow/tensorflow)  

以下我要講解的是 [Daniel Nouri's Tutorial](http://danielnouri.org/notes/2014/12/17/using-convolutional-neural-nets-to-detect-facial-keypoints-tutorial/#second-model-convolutions) (但我把它改成了 Tensorflow 版)
該 tutorial 寫得非常詳盡。
有興趣的人可以翻出來看 (它是用 Theano/Lassagne 做的)。


# Computational Graph
在開始之前，我們得先說明 symbolic operation 以及 computational graph 的概念。

Tensorflow 其實是在建立一張 computational graph of tensors，讓 data 可以 flow
以使 model 可以最終被 data 給塑型（故名 Tensorflow）。

Computational graph 是 Tensorflow 提供的一種特殊的數學模組。
(我的理解是)它把 symbolic operation 和 numerical operation 分開，
讓你可以不用親手去導 gradient、不用 assign update rule。
在建立 graph 的階段，computational graph 中的 symbolic operations 不會*立即*產生結果。
相反的，它只是把變數和運算連結起來而已。
一張典型的 computational graph 如下圖所示：
![img](imgs/compugraph1.png)
本圖中：x (最底下的)經過一次 hidden layer，然後再經過 output layer，最後和 supervised target (y) 計算了 objective。  
  
  
放大其中一個模組：
![img](imgs/compugraph2.png)
在hidden layer 內，x 和 w 進行矩陣乘法，然後和 bias term 相加。最後再經過 ReLU 出去。  


舉例來說，在 MATLAB中
```octave
% Matlab
x = ones(128, 5)
y = x + 1.0
```  
最後一行的是 numerical operation，它會讓你立即得到 128x5 個 2.0

然而在 Tensorflow 中
```python
# Tensorflow
x = tf.ones(128, 5)
y = tf.add(x, 1.0)
```
最後一行其實只是 computational graph 的 construction/definition
你只會得到一個 symbolic variable，名叫 y。
它還不會真正有值。
除非你**啟動** graph (也就是呼叫 Session 物件，後面會講)。

### Remark
我用一個比喻來形容 computational graph：
graph 中你所設計的 model parameters 就像是一塊大理石 (一開始它很奇形怪狀)，
而 data 就像是瀑布的水或米開朗基羅的雕刻刀。
Cost function 就像是你對作品的期許。
作為雕刻刀的 data 不斷地銷過作為大理石的 parameters 的表面，最後你就會得到你的作品。


# 範例題目：[Facial Keypoint Detection](https://www.kaggle.com/c/facial-keypoints-detection)
給一張 96x96 的人臉，是否能偵測出臉部的15個特徵的座標？（詳述請看官網 link）  
我接下來會用這個範例題講解：如何用 Tensorflow 做出簡單的模型 (NN 和 CNN)  


### Prerequisite
1. (在開始之前，請到註冊一個Kaggle帳號並[下載](https://www.kaggle.com/c/facial-keypoints-detection)dataset。
2. 需要 numpy, tensorflow, (matplotlib), pandas (讀 csv 檔)


### Optional
1. 你可以透過執行 convert_csv_to_png 來看 dataset 的圖。

首先，我們來建立一個 fully-connected NN with 1 hidden layer

In [ ]:
import tensorflow as tf

## 從寫 Fully-Connected Layer 入Tensorflow 的門
Tensorflow 麻煩的地方就是：它沒有已經寫好的 fully-connected layer API  
所以我們得自己寫一個；這樣以後就可以一直用了。

In [ ]:
def full_layer(x, fan_out, layer_name, nonlinear=tf.nn.relu):
    ''' Fully-connected layer    '''
    with tf.variable_scope(layer_name):
        fan_in = x.get_shape().as_list()[-1]    # 這就是我痛恨 TF 的地方...之一
        shape = (fan_in, fan_out)
        w = tf.get_variable('w', initializer=glorot_init(shape))
        b = tf.get_variable('b', initializer=tf.zeros([fan_out]))
        xw = tf.matmul(x, w) # 矩陣乘法
        o = tf.add(xw, b)    # 加法。dimension 不一樣的時候，會分配到適合的維度做運算
        y = nonlinear(o)
    return y

我解釋一下 input arguments：  
`x`: 我們預期他是一個 matrix，進 full_layer 後要做 xw 的運算 (所以請注意他的 dimension)   
`fan_out`: output 要幾個 node
`layer_name`: 這個 layer 的名字 (Namespace)。下一段會講這個設計的用意。事實上你大可以不這樣寫。

### Variable Scoping (Namespace)
```python
with tf.variable_scope(layer_name):
```
意思是：以下宣告的變數全部屬於這個命名空間。  
這樣做的好處是：我們可以一直用相同的變數名──只要他們屬於不同的namespace。  

例如我可以這樣寫
```python
x = tf.Variable(name='x', shape=(128, 5))

with tf.variable_scope('hidden9'):
    w = tf.get_variable('w', initializer=tf.ones((5, 3)))
    x = tf.matmul(x, w)
    
with tf.variable_scope('output9'):
    w = tf.get_variable('w', initializer=tf.ones((3, 1)))
    x = tf.matmul(x, w)
```

如此一來，我就可以一直用 w 來指稱新的變數。  
這一開始會顯得很詭異。但別忘了，Tensorflow 在一開始的階段都是 symbolic operation，目的只是要建立 computational graph。也就是說，目的是把整個 flow 建立起來。  
所以所有等號左邊的東西都只是**暫時的**。~~(啊~ 我的變數業障重啊!)~~

### Get the Shape of a Tensorflow
```python
fan_in = x.get_shape().as_list()[-1]
```
這是我不太喜歡TF的一個地方：
2. 要用很不直覺的方法取得 shape
1. 他連 shape 都是 tensor，所以又要用另一個 method 把他變成 list
但如果我們非得取得 shape，那就只能這樣寫了

### Operands (變數)
接下來才是重點：怎麼宣告 Tensorflow 的「變數」
在 TF 中，實際上只有透過兩個方式產生的東西才叫做「變數(tf.Variable object)」

1. 直接呼叫 `tf.Variable` class
```python
x = tf.Variable(name='x', initial_value=tf.ones(shape=(5, 3)))
```

2. 透過變數管理員 `tf.get_variable` 來指定變數
```python
x = tf.get_variable(name='x', initializer=tf.ones(shape=(5, 3)))
```

我個人偏好使用後者。因為他會在撞名的時候就跟你回報錯誤。  
另請注意：
1. 如果用 `tf.Variable` 宣告變數，`name` atrribute 不一定要填。(但我建議還是填吧)
2. 兩個方式都要填入 initial_value。可以填入 Numpy array，也可以填 Tensorflow tensor。我建議用後者。
3. 在 `full_layer` 函式中，我用了另一個自訂的初始化函式 `glorot_init`，詳細定義請參考 util.py


再提醒一次：由於這邊定義的是 symbolic variable  
所以不能**直接**取值。  
例如，`print x` 只會告訴你說 x 是一個 Variable。沒了。他不會告訴你他的值是多少。  
x 是一個變數。它在 training 的過程中，值會不斷被更新。  
後文會講解如何取得 `tf.Variable` 的值。

### Operations (運算)
有了變數 (x, w, b)，接下來就是做運算了。  
這邊很直觀，沒什麼可講的。  
至於有什麼運算可做，請去查API。大抵上就是矩陣加減乘除。  

```python
xw = tf.matmul(x, w) # 矩陣乘法
o = tf.add(xw, b)    # 加法。dimension 不一樣的時候，會分配到適合的維度做運算
y = nonlinear(o)
```

Hints:
1. 如果你的矩陣乘法需要轉置，請在 input argument 裡面指定。例如 `transpose_a=True`
2. 請注意：雖然 TF 有實作 operator overloading，但我建議不要使用。  
例如上面的  
`o = tf.add(xw, b)`  
最好別寫成  
`o = xw + b`


## 小結
恭喜，現在你基本上已經能寫 Tensorflow 了。
但我們還沒講要怎麼讓資料 flow through the computational graph
下一章我們就會實作一個 FC-NN with 1-hidden layer
屆時就會講到 training 的部分。